# Task 3 - Feature Engineering

In [1]:
import numpy as np
from datasets import Dataset, ClassLabel, Sequence
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append(os.path.abspath("../src"))
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("..")
print(os.getcwd()) 

d:\KAIM 5,6,7\Week-5\Credit-Risk-Probability-Model


In [8]:
from  data_processing import feature_engineering

In [9]:
# Load your data
df = pd.read_csv("Data/raw/data.csv")

processed_df = feature_engineering(df)
if processed_df is not None:
    print(processed_df.head())
else:
    print("🚨 Feature engineering failed.")


 Error during feature engineering: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'
🚨 Feature engineering failed.
